In [1]:
## code from https://github.com/ankanbhunia/Handwriting-Transformers

# Run below code to set up the environment
import os
repo_path = 'Handwriting-Transformers'
if not os.path.exists(repo_path) and os.getcwd().split('/')[-1] != repo_path:
    !git clone https://github.com/ankanbhunia/Handwriting-Transformers
if os.getcwd().split('/')[-1] != repo_path:
    %cd Handwriting-Transformers
if not os.path.exists('files'): # Get the model and data files
    %pip install --upgrade --no-cache-dir gdown
    !gdown --id 16g9zgysQnWk7-353_tMig92KsZsrcM6k
    !unzip files.zip && rm files.zip

c:\Users\Anita Dash\Music\handwriting-generator-model\Code\HWT-model\Handwriting-Transformers


Cloning into 'Handwriting-Transformers'...


C:\Users\Anita Dash\anaconda3\lib\site-packages\gdown\cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (uriginal): https://drive.google.com/uc?id=16g9zgysQnWk7-353_tMig92KsZsrcM6k
From (redirected): https://drive.google.com/uc?id=16g9zgysQnWk7-353_tMig92KsZsrcM6k&confirm=t&uuid=5419dda7-32eb-42ed-a798-39f77043c8c4
To: c:\Users\Anita Dash\Music\handwriting-generator-model\Code\HWT-model\Handwriting-Transformers\files.zip

  0%|          | 0.00/685M [00:00<?, ?B/s]
  0%|          | 524k/685M [00:01<25:02, 455kB/s]
  0%|          | 1.57M/685M [00:01<07:58, 1.43MB/s]
  0%|          | 2.62M/685M [00:01<04:48, 2.37MB/s]
  1%|          | 3.67M/685M [00:01<03:19, 3.42MB/s]
  1%|          | 4.72M/685M [00:01<02:37, 4.33MB/s]
  1%|          | 6.29M/685M [00:01<01:58, 5.71MB/s]
  1%|          | 7.34M/685M [00:02<01:44, 6.47MB/s]
  1%|          | 8.39M/685M [

In [2]:
os.chdir('../HWT-model/Handwriting-Transformers/')

In [53]:
# Import the libraries
import os
from data.dataset import TextDataset, TextDatasetval
import torch
import cv2
import numpy as np
from models.model import TRGAN
from params import *
from torch import nn
from data.dataset import get_transform
import pickle
from PIL import Image
import tqdm

In [54]:
model_path = 'files/iam_model.pth'

In [59]:
writer_img_paths = ['../DataSet/anita-1/', '../DataSet/dhruv-1/', '../DataSet/shambu-1/', '../DataSet/taha-1/'] # For custom writer styles
writer_names = ['Anita', 'Dhruv', 'Shambu', 'Taha'] # For custom writer styles
text = 'Magic madness heaven sin Saw you there and I thought'

batch_size = 4 # Number of results per page (Change in params.py also)
output_path = '../demo_output/'

# Remove files from the output folder (Not folders)
for f in os.listdir(output_path):
    if os.path.isfile(os.path.join(output_path, f)):
        os.remove(os.path.join(output_path, f))
os.makedirs(output_path, exist_ok=True)

In [55]:
def preprocess_writer_images(image_path):
    '''Opens folder of images and returns a list of word images resized to (Wx32)'''
    image_list = []
    img_ht = 32
    for image_name in os.listdir(image_path):
        image = cv2.imread(os.path.join(image_path, image_name))
        image[image>150] = 255
        image = cv2.resize(image, (image.shape[1]*img_ht//image.shape[0], img_ht))
        image_list.append((Image.fromarray(image), image_name))
    return image_list

def get_word_images(paragraph_img_path):
    '''Opens a paragraph image and returns a list of word images'''
    # TODO
    pass

def post_process(result_img, threshold=0.8):
    '''Remove grey background from words'''
    result_img[result_img>threshold] = 1
    img_height, _ = result_img.shape
    # Find the rightmost column with some black pixels, and crop the image there
    col_sums = np.sum(result_img, axis=0)
    boolean_array = col_sums < img_height
    rightmost_col = len(boolean_array) - np.argmax(boolean_array[::-1]) - 1
    result_img = result_img[:, :rightmost_col+5]
    return result_img

In [56]:
# Create the pickle file for input handwriting style

# Create test dictionary
test_dataset = {'test': {}}
for writer_id, writer_img_path in enumerate(writer_img_paths):
    test_dataset['test'][writer_id] = []
    for word_img, label in preprocess_writer_images( writer_img_path):
        test_dataset['test'][writer_id].append({'img': word_img, 'label': label})

# Save the pickle file
with open('../DataSet/demo.pickle', 'wb') as f:
    pickle.dump(test_dataset, f)
data_path = '../DataSet/demo.pickle'

In [57]:
# Load the model and the dataset
print ('(1) Loading dataset files...')
TextDatasetObjval = TextDatasetval(base_path = data_path, num_examples = 15)
datasetval = torch.utils.data.DataLoader(
            TextDatasetObjval,
            batch_size=batch_size,
            shuffle=True,
            num_workers=0,
            pin_memory=True, drop_last=True,
            collate_fn=TextDatasetObjval.collate_fn)

print ('(2) Loading model...')

model = TRGAN()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cuda:0')
model.netG.load_state_dict(torch.load(model_path, map_location=device))
print (model_path+' : Model loaded Successfully')

print ('(3) Loading text content...')
text_encode =  [j.encode() for j in text.split(' ')]
eval_text_encode, eval_len_text = model.netconverter.encode(text_encode)
eval_text_encode = eval_text_encode.to(device).repeat(batch_size, 1, 1)

(1) Loading dataset files...
(2) Loading model...


c:\python\Python39\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\python\Python39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


initialize network with N02
initialize network with N02
initialize network with N02


c:\python\Python39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
c:\python\Python39\lib\site-packages\torchvision\models\inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


files/iam_model.pth : Model loaded Successfully
(3) Loading text content...


In [61]:
input_handwriting_style_imgs = []
output_imgs = []

# Generate the images
for i,data_val in enumerate(tqdm.tqdm(datasetval)): 
    generated_imgs = model.netG.Eval(data_val['simg'].to(device), eval_text_encode)

for j in range(len(writer_img_paths)):
    os.makedirs(output_path + 'writer_' + writer_names[j], exist_ok=True)
    folder_path = output_path + 'writer_' + writer_names[j]
    for i in range(len(generated_imgs)):
        img = post_process(generated_imgs[i][j].squeeze().cpu().detach().numpy())
        # save file in new made directory
        path = folder_path + '/' + str(i) + '.png'
        cv2.imwrite(path, img*255)


100%|██████████| 1/1 [00:01<00:00,  1.79s/it]
